# Plant CV hands-on

Everything is running on a web server so you don't need to install anything.

To run a block of code, click on the code block, hold [shift] and press [enter]

For more information check out the plantcv website: https://plantcv.danforthcenter.org 

and the documentation: https://plantcv.readthedocs.io/en/stable/

There are also many more usefull tutorials here: https://plantcv.readthedocs.io/en/stable/tutorials/



In [ ]:
# Import Libraries 
from plantcv import plantcv as pcv
import matplotlib
import numpy as np

In [ ]:
class options:
    def __init__(self):
        #self.image = "./img/original_image.jpg"
        self.image = "images/2021_08_19_12_44_11-11-71-VBCF_5x4_028-RGB2-FishEyeCorrected.png"
        self.debug = "plot"
        self.writeimg= False
        self.result = "vis_tutorial_results.json"
        self.outdir = "." # Store the output to the current directory
        
# Get options
args = options()

# Set debug to the global parameter 
pcv.params.debug = args.debug

In [ ]:
# Read image

# Inputs:
#   filename - Image file to be read in 
#   mode - How to read in the image; either 'native' (default), 'rgb', 'gray', or 'csv'
img, path, filename = pcv.readimage(filename=args.image)

In [ ]:
mask, masked_img = pcv.threshold.custom_range(img=img, lower_thresh=[0,0,136], upper_thresh=[255,132,255], channel='LAB')


In [ ]:
# Convert mask to binary

masked_img.shape

bin_img = np.clip(masked_img.copy()[:,:,0],0,1)
matplotlib.pyplot.imshow(bin_img)


In [ ]:
a_fill = pcv.fill(bin_img=bin_img, size=200)

In [ ]:
# Inputs:
#   img  = input image
#   mask = a binary mask used to detect objects
obj, obj_hierarchy = pcv.find_objects(img=img, mask=a_fill)

In [ ]:
# Inputs:
#   img     = input image
#   coord   = top left coordinate to begin the ROI grid
#   radius  = radius for each ROI
#   spacing = spacing between each ROI
#   nrows   = number of rows in the ROI grid
#   ncols   = number of columns in the ROI grid
rois, roi_hierarchy = pcv.roi.multi(img=img, coord=(730,520), radius=200, 
                                    spacing=(670, 670), nrows=4, ncols=5)

In [ ]:
# Inputs:
#   start = beginning value for range
#   stop  = ending value for range (exclusive)
plant_ids = range(0, len(rois))


In [ ]:
# Create a copy of the original image for annotations

# Inputs:
#   img = rgb image
img_copy = np.copy(img)

# Set debug to None (plotting all of the images would be very verbose output)
pcv.params.debug = None

# Create a for loop to interate through every ROI (plant) in the image
for i in range(0, len(rois)):
    # The ith ROI, ROI hierarchy, and plant ID
    roi = rois[i]
    hierarchy = roi_hierarchy[i]
    plant_id = plant_ids[i]
    # Subset objects that overlap the ROI
    # Inputs:
    #   img            = input image
    #   roi_contour    = a single ROI contour
    #   roi_hierarchy  = a single ROI hierarchy
    #   object_contour = all objects detected in a binary mask
    #   obj_hierarchy  = all object hierarchies
    #   roi_type       = "partial" (default) keeps contours that overlap
    #                    or are contained in the ROI. "cutto" cuts off
    #                    contours that fall outside the ROI. "largest"
    #                    only keeps the largest object within the ROI
    plant_contours, plant_hierarchy, mask, area = pcv.roi_objects(img=img, 
                                                                  roi_contour=roi, 
                                                                  roi_hierarchy=hierarchy, 
                                                                  object_contour=obj, 
                                                                  obj_hierarchy=obj_hierarchy, 
                                                                  roi_type="partial")

    # If the plant area is zero then no plant was detected for the ROI
    # and no measurements can be done
    if area > 0:
        # Combine contours together for each plant
        # Inputs:
        #   img       = input image
        #   contours  = contours that will be consolidated into a single object
        #   hierarchy = the relationship between contours
        plant_obj, plant_mask = pcv.object_composition(img=img, 
                                                       contours=plant_contours, 
                                                       hierarchy=plant_hierarchy)        
        # Analyze the shape of each plant
        # Inputs:
        #   img   = input image
        #   obj   = composed object contours
        #   mask  = binary mask that contours were derived from
        #   label = a label for the group of measurements (default = "default")
        img_copy = pcv.analyze_object(img=img_copy, obj=plant_obj, 
                                      mask=plant_mask, label="plant_" + str(plant_id))

pcv.plot_image(img_copy)

In [ ]:
pcv.print_results(filename='results.txt')

In [ ]:
pcv.outputs.save_results(filename='results.csv', outformat = 'csv')